You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Content Plan: 
Title: Unraveling the Latest Trends in Artificial Intelligence

Outline:
I. Introduction
- Definition of Artificial Intelligence
- Importance of AI in today's world
II. Latest Trends in Artificial Intelligence
- Machine learning advancements
- Natural language processing developments
- AI in healthcare and finance sectors
- Ethical considerations in AI
III. Key Players in the AI Industry
- Google
- Amazon
- IBM
- Microsoft
IV. Noteworthy News in Artificial Intelligence
- Recent breakthroughs in 

I now can give a great answer

Final Answer:
# Unraveling the Latest Trends in Artificial Intelligence

## Introduction
Artificial Intelligence, often referred to as AI, is the simulation of human intelligence processes by machines, especially computer systems. In today's fast-paced world, AI plays a crucial role in various sectors, revolutionizing the way we live and work. From enhancing business operations to improving healthcare outcomes, AI has become an indispensable tool for innovation and growth.

## Latest Trends in Artificial Intelligence
The field of AI is constantly evolving, with new trends shaping the landscape of technology. Machine learning advancements have propelled AI to new heights, allowing machines to learn from data and improve their performance over time. Natural language processing developments have also made significant strides, enabling machines to understand and process human language more effectively. In sectors like healthcare and finance, AI is being used 

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# Unraveling the Latest Trends in Artificial Intelligence

## Introduction
Artificial Intelligence, often referred to as AI, is the simulation of human intelligence processes by machines, especially computer systems. In today's fast-paced world, AI plays a crucial role in various sectors, revolutionizing the way we live and work. From enhancing business operations to improving healthcare outcomes, AI has become an indispensable tool for innovation and growth.

## Latest Trends in Artificial Intelligence
The field of AI is constantly evolving, with new trends shaping the landscape of technology. Machine learning advancements have propelled AI to new heights, allowing machines to learn from data and improve their performance over time. Natural language processing developments have also made significant strides, enabling machines to understand and process human language more effectively. In sectors like healthcare and finance, AI is being used to streamline processes, improve efficiency, and enhance decision-making. However, ethical considerations in AI, such as bias in algorithms and data privacy concerns, continue to be hot topics of discussion in the industry.

## Key Players in the AI Industry
Leading tech companies like Google, Amazon, IBM, and Microsoft are at the forefront of AI innovation. These key players invest heavily in research and development, driving advancements in AI technologies and applications. Their contributions to the field have shaped the way we interact with AI on a daily basis, from virtual assistants to predictive analytics tools.

## Noteworthy News in Artificial Intelligence
Recent breakthroughs in AI research have paved the way for exciting possibilities in various industries. From autonomous vehicles to personalized medicine, AI applications are transforming the way we live and work. However, the impact of AI on the job market remains a topic of debate, with concerns about job displacement and the need for upskilling in the workforce.

## Target Audience Analysis
This blog post is tailored for tech enthusiasts, business professionals, students interested in AI, and individuals curious about the future of technology. By exploring the latest trends, key players, and noteworthy news in AI, readers will gain valuable insights into the rapidly evolving world of artificial intelligence.

## SEO Keywords
Artificial Intelligence trends, AI key players, Latest news in AI, Artificial Intelligence developments

## Call to Action
I encourage readers to explore AI further through online courses or workshops, share their thoughts on AI trends and developments, and access relevant resources for continued learning. Stay informed and inspired by the latest trends in Artificial Intelligence!

In conclusion, Artificial Intelligence continues to shape the future of technology, with new trends, key players, and groundbreaking news driving innovation and growth in the industry. Stay ahead of the curve and embrace the possibilities of AI in today's world.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

In [ ]:
Markdown(result)

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).